In [1]:
import calendar
import time
import datetime
import pandas as pd
import numpy as np
from itertools import compress

# 排课

## 定义课节

In [4]:
python_1on1 = Classes('Python 1on1', duration=90)

## 检测老师可排课时间

In [7]:
start_1 = '2019-02-03 19:00'
end_1 = '2019-02-03 21:00'
start_2 = '2019-02-05 19:00'
end_2 = '2019-02-05 21:00'
available = [(start_1, end_1), (start_2, end_2)]
available = [tuple(map(parse_time, (a, b))) for (a, b) in available]
# parse_time(end_1).strftime('%Y-%m-%d %H:%M')
# [tuple(map(parse_time, (a, b))) for (a, b) in []]

In [8]:
check_available(available, (datetime.datetime(2019, 2, 3, 19,20), datetime.datetime(2019, 2, 3, 20,0)))
check_conflict(available, (datetime.datetime(2019, 2, 3, 19,20), datetime.datetime(2019, 2, 3, 20,0)), 10)

([True, False],
 [(datetime.datetime(2019, 2, 3, 19, 0),
   datetime.datetime(2019, 2, 3, 21, 0))])

## 定义讲师

In [17]:
class Lecturer:
    def __init__(self, name, available, min_per_day, rest = 10, schedule = {}):
        '''available: list of tuples enveloping start time and end time'''
        self.name = name
        self.__reg_available = available.copy()
        self.available = available
        self.rest = rest
        self.min_per_day = min_per_day
        self.schedule = schedule
    def get_reg_available(self):
        return self.__reg_available
    
    def init_appointment(self, your_class, student, start):
        self.classes = your_class
        available_parse = [tuple(map(parse_time, (a, b))) for (a, b) in self.available]
        proposal = (parse_time(start), parse_time(start) + datetime.timedelta(minutes = your_class.duration))
        availability, occupation = check_available(available_parse, proposal)
        if any(availability):
            previous = np.concatenate(list(self.schedule.values())) if bool(self.schedule) else list(self.schedule.values())
            previous = [tuple(map(parse_time, (a, b))) for (a, b) in previous]
            conflict, conf_class = check_conflict(previous, proposal, self.rest)
            if any(conflict):
                conf_class = tuple(map(lambda x: x.strftime('%Y-%m-%d %H:%M'), conf_class))
                print('老师在{}-{}这段时间内有课，而且请给老师{}分钟休息时间，另行安排!'.format(conf_class[0], conf_class[1], self.rest))
            else:
                end = parse_time(start) + datetime.timedelta(minutes = self.classes.duration)
                print('没问题，该时间段内老师可排课！')
                if student.name not in self.schedule.keys():
                    self.schedule[student.name] = [(start, end.strftime('%Y-%m-%d %H:%M'))]
                else:
                    self.schedule[student.name].extend([(start, end.strftime('%Y-%m-%d %H:%M'))])
                # 剩余时间
                available_parse = [tuple(map(parse_time, (a, b))) for (a, b) in self.available]
                for i in occupation:
                    insertee = [(available_parse[i][0], parse_time(start)),
                                (end, available_parse[i][1])]
                    insertee = list(filter(lambda x: x[1] > x[0], insertee))
                    insertee = [tuple(map(lambda x: x.strftime('%Y-%m-%d %H:%M'), (a, b))) for (a, b) in insertee]
                    del self.available[i]
                    self.available[i:i] = insertee
        else:
            print('老师有安排了，请考虑以下时间段：\n{}'.format(self.available))
    def change_appointment(self, student, which, prop_start):
        prop_end = parse_time(prop_start) + datetime.timedelta(minutes = self.classes.duration)
        available_parse = [tuple(map(parse_time, (a, b))) for (a, b) in self.available]
        availability, occupation =  check_available(available_parse, (parse_time(prop_start), prop_end))
        if any(availability):
            previous = np.concatenate(list(self.schedule.values())) if bool(self.schedule) else list(self.schedule.values())
            previous = [tuple(map(parse_time, (a, b))) for (a, b) in previous]
            conflict, conf_class = check_conflict(previous, (parse_time(prop_start), prop_end), self.rest)
            conf_class = tuple(map(parse_time, conf_class))
            if any(conflict):
                print('老师在{}-{}这段时间内有课，而且请给老师{}分钟休息时间，另行安排!'.format(conf_class[0], conf_class[1], self.rest))
            else:
                print('没问题，该时间段内老师可排课！')
                self.available.append(self.schedule[student.name][which])
                self.available = sorted([x for x in self.available])           
                while any([self.available[i][0] == self.available[i-1][1] for i in range(1, len(self.available))]):
                    del_idx = list(filter(lambda i: self.available[i][0] == self.available[i-1][1], range(1, len(self.available))))
                    idx = list(filter(lambda i: not self.available[i][0] == self.available[i-1][1], range(1, len(self.available))))
                    idx.append(0)
                    for i in del_idx:
                        self.available[i-1] = (self.available[i-1][0], self.available[i][1])
                    self.available = sorted([self.available[x] for x in idx])
    
                self.schedule[student.name][which] = (prop_start, prop_end.strftime('%Y-%m-%d %H:%M'))

        else:
            print('老师有安排了，请考虑以下时间段：\n{}'.format(self.available))

## 定义学生

In [10]:
class Student:
    def __init__(self, name, available, schedule = {}):
        self.name = name
        self.__reg_available = available.copy()
        self.available = available
        self.schedule = schedule
    def find_lecturer(self, lecturers):
        pass

定义讲师李海

In [122]:
lecturer_names = ['蔡海荣', '韩雷', '付杰', '杨波', '郑斌帆', '贾胜杰', '李海', '王潇宇', '丘桦东']
lecturers = {}
for lect in lecturer_names:
    lecturers[lect] = pd.read_excel('../../Python_1on1/Lecturers_available_time.xlsx', sheet_name=lect, index_col=0)\
    .drop(['周', '每日空闲时间', '最少可排课时'], axis = 0).drop(['Unnamed: 14'], axis = 1)
# pd.read_excel('../../Python_1on1/Lecturers_available_time.xlsx', sheet_name='韩雷').drop([0, 16], axis = 0)

In [123]:
# start = [x.split('-')[0] for x in list(lecturers['韩雷'].drop(17).iloc[:, 0])]
# dates = [str(x) for x in list(lecturers['韩雷'].drop(17).columns[1:])][:-1]
# dates = [x.split(' ')[0] for x in dates]
# pd.melt(lecturers['韩雷'].drop(17))
# lecturers['韩雷'].drop(17).index = lecturers['韩雷'].drop(17).columns[1:]
lecturers['韩雷']

,2019-03-12 00:00:00,2019-03-13 00:00:00,2019-03-14 00:00:00,2019-03-15 00:00:00,2019-03-16 00:00:00,2019-03-17 00:00:00,2019-03-18 00:00:00,2019-03-19 00:00:00,2019-03-20 00:00:00,2019-03-21 00:00:00,2019-03-22 00:00:00,2019-03-23 00:00:00,2019-03-24 00:00:00
（优先2）日期,,,,,,,,,,,,,
8:00-9:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9:00-10:00,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1
10:00-11:00,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1
11:00-12:00,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1
12:00-13:00,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1
13:00-14:00,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1
14:00-15:00,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1
15:00-16:00,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1
16:00-17:00,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1


In [117]:
Lihai = Lecturer(name = '李海', available = [(start_1, end_1),
                                             (start_2, end_2)])
print('讲师姓名：{}'.format(Lihai.name))
print('讲师可排课时间：{}'.format(Lihai.available))

TypeError: __init__() missing 1 required positional argument: 'min_per_day'

定义学生李白

In [95]:
LiBai = Student('李白', [('2019-02-03 14:00', '2019-02-03 18:00'),
                         ('2019-02-03 20:00', '2019-02-03 22:00'),
                         ('2019-02-05 20:00', '2019-02-05 22:00')])

定义学生杜甫

In [96]:
DuFu = Student('杜甫', [('2019-02-03 20:00', '2019-02-03 22:00'),
                         ('2019-02-05 14:00', '2019-02-05 16:00'),
                         ('2019-02-05 20:00', '2019-02-05 22:00')])

初次预约课程。

In [97]:
Lihai.init_appointment(python_1on1, LiBai, start= '2019-02-03 19:30')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(Lihai.name, Lihai.schedule))
print('老师的剩余可用时间为\n{}'.format(Lihai.available))

没问题，该时间段内老师可排课！
李海老师，你的课程表如下：
{'李白': [('2019-02-03 19:30', '2019-02-03 20:10')]}
请提前做好准备！
老师的剩余可用时间为
[('2019-02-03 19:00', '2019-02-03 19:30'), ('2019-02-03 20:10', '2019-02-03 21:00'), ('2019-02-05 19:00', '2019-02-05 21:00')]


给第二个学生预约课程：

In [98]:
Lihai.init_appointment(python_1on1, DuFu, start= '2019-02-03 20:20')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(Lihai.name, Lihai.schedule))
print('老师的剩余可用时间为\n{}'.format(Lihai.available))

没问题，该时间段内老师可排课！
李海老师，你的课程表如下：
{'李白': [('2019-02-03 19:30', '2019-02-03 20:10')], '杜甫': [('2019-02-03 20:20', '2019-02-03 21:00')]}
请提前做好准备！
老师的剩余可用时间为
[('2019-02-03 19:00', '2019-02-03 19:30'), ('2019-02-03 20:10', '2019-02-03 20:20'), ('2019-02-05 19:00', '2019-02-05 21:00')]


杜同学要排第二节课

In [99]:
Lihai.init_appointment(python_1on1, DuFu, start= '2019-02-05 20:20')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(Lihai.name, Lihai.schedule))
print('老师的剩余可用时间为\n{}'.format(Lihai.available))

没问题，该时间段内老师可排课！
李海老师，你的课程表如下：
{'李白': [('2019-02-03 19:30', '2019-02-03 20:10')], '杜甫': [('2019-02-03 20:20', '2019-02-03 21:00'), ('2019-02-05 20:20', '2019-02-05 21:00')]}
请提前做好准备！
老师的剩余可用时间为
[('2019-02-03 19:00', '2019-02-03 19:30'), ('2019-02-03 20:10', '2019-02-03 20:20'), ('2019-02-05 19:00', '2019-02-05 20:20')]


杜同学要调课

In [100]:
Lihai.change_appointment(DuFu, 0, '2019-02-05 19:00')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(Lihai.name, Lihai.schedule))
print('老师的剩余可用时间为\n{}'.format(Lihai.available))

没问题，该时间段内老师可排课！
李海老师，你的课程表如下：
{'李白': [('2019-02-03 19:30', '2019-02-03 20:10')], '杜甫': [('2019-02-05 19:00', '2019-02-05 19:40'), ('2019-02-05 20:20', '2019-02-05 21:00')]}
请提前做好准备！
老师的剩余可用时间为
[('2019-02-03 19:00', '2019-02-03 19:30'), ('2019-02-03 20:10', '2019-02-03 21:00'), ('2019-02-05 19:00', '2019-02-05 20:20')]


李白同学的课程表：

In [101]:
LiBai.schedule = Lihai.schedule[LiBai.name]
LiBai.schedule

[('2019-02-03 19:30', '2019-02-03 20:10')]

杜甫同学的课表：

In [103]:
DuFu.schedule = Lihai.schedule[DuFu.name]
DuFu.schedule

[('2019-02-05 19:00', '2019-02-05 19:40'),
 ('2019-02-05 20:20', '2019-02-05 21:00')]

## TBD

* 如何匹配合适的老师？

* 如何找到一段合适的上课时间？

In [ ]:
class Schedule:
    def __init__(self, year, month):
        self.year = year
        self.month = month
    def term(self, lenghth):
        cale = calendar.Calendar()
        self.term_length = lenghth
        month_span = self.term_length // 28 +1
        schedule = cale.monthdatescalendar(year=self.year, month=self.month)
        schedule = pd.DataFrame(schedule,columns = calendar.weekheader(6).strip().split('    ')).iloc[:4]
#         schedule = pd.DataFrame(columns = calendar.weekheader(6).strip().split('    '))
        for i in range(1,month_span):
            schedule_i = cale.monthdatescalendar(year=self.year, month=self.month + i)
            schedule_i = pd.DataFrame(schedule_i,columns = calendar.weekheader(6).strip().split('    '))
            schedule = schedule.append(schedule_i)
            schedule.reset_index(drop=True, inplace=True)
        return schedule

In [ ]:
Schedule(2019, 1).term(124)

In [ ]:
18*7

In [ ]:
class Lecturer:
    def __init__(self, name, available):
        self.name = str(name),
        self.available = available
    def end(self, duration):
        '''calculate where his class will ends'''
        ends = []
        for i in self.available:
            ends.append(i+ datetime.timedelta(minutes = duration))
        return ends
    def assign(self, schedual):
        '''assign his available time to te schedual'''
        for i in self.available:
            date = i.year
            loc = schedual.loc('')

In [ ]:
Lihai = Lecturer(name='李海', available=[datetime.datetime(2019, 2, 3, 19, 0), datetime.datetime(2019, 2, 5, 10, 0)])
datetime.datetime(2019, 1, 23, 19, 0, 0)

schedual['Mon'][1] == `datetime.datetime(2019, 2, 4)

In [1]:
url = 'https://docs.google.com/spreadsheets/d/1SsJ8IXLczjE8JTkUDFLCzSe66agAA7Rt0RA6hFbJcZ4/export?format=csv&gid=585202811'

In [2]:
import pandas as pd
pd.read_csv(url).head()

ParserError: Error tokenizing data. C error: Expected 1 fields in line 6, saw 2


# 尝试群发学习计划

In [ ]:
import pandas as pd
import itchat as ic

In [ ]:
ic.auto_login()

获取单个用户的ID

In [ ]:
userID = ic.search_friends(name='Udacity-Tyelcie')[0]['UserName']
userID

获取群ID

In [ ]:
chatroomID = ic.search_chatrooms('测试')[0]['UserName']
chatroomID

发文字消息

In [ ]:
ic.send('https://shimo.im/sheet/xwJ8FkJhUcUKeXyM/', toUserName=userID)

发图片消息

In [ ]:
ic.send_image('../stobserver/test.jpg', toUserName=chatroomID)

## 简单粗暴地抓取石墨文档表格

把要抓的内容选中，Ctrl+C，然后用`pd.read_clipboard()`从剪贴板读取数据。

In [ ]:
students = pd.read_clipboard(sep='\t')
students.head(3)

如果要群发消息，注意修改任务序列和消息模板：

In [ ]:
task_seq = 'Task 1'
msg_template = 'Hello，{}同学，接下来三天你需要完成的任务是：\n\n{}'
def send_task_group(task_seq, msg_template, students_df):
    for i in range(1, len(students_df)):
        student = students_df['姓名'][i]
        userID = ic.search_friends(name=student)[0]['UserName']
        personal_task = students_df[task_seq][i]
        msg = msg_template.format(student, personal_task)
        ic.send(msg, userID)

In [ ]:
task_seq = 'Task 1'
msg_template = 'Hello，{}同学，接下来三天你需要完成的任务是：\n\n{}'
send_task_group(task_seq, msg_template, students)

如果需要单独发消息，注意修改学生姓名、任务序列和消息模板：

In [ ]:
student = 'Udacity-Tyelcie'
task_seq = 'Task 1'
msg_template = 'Hello，{}同学，接下来三天你需要完成的任务是：\n\n{}'

def send_task_personal(student_name, task_seq, msg_template, students_df):
    stu_pos = list(filter(lambda x: students_df['姓名'][x] == student_name, students_df.index.tolist()))[0]
    msg = msg_template.format(student_name, students_df[task_seq][stu_pos])
    userID = ic.search_friends(name=student_name)[0]['UserName']
    ic.send(msg, userID)

In [ ]:
send_task_personal('Udacity-Tyelcie', task_seq, msg_template, students)